# 1. Unperturbed Earth-orbiting Satellite

In [21]:
%%timeit

import numpy as np                          # The go-to Python for matrices.
from tudatpy import constants               # Module to retrieve constants in tudatpy.
from tudatpy import elements                # Module for state conversions.
from tudatpy import io                      # Module for saving to text files.
from tudatpy import ephemerides             # Module for ephemeris data.
from tudatpy import interpolators
from tudatpy import numerical_integrators
from tudatpy import spice_interface
from tudatpy import basic_astrodynamics
# from tudatpy import orbital_element_conversions # LEGACY MODULE
from tudatpy import propagators
from tudatpy import aerodynamics
from tudatpy import simulation_setup

15.4 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## General Simulation Parameters

In [18]:
%%timeit

# Load spice kernels.
spice_interface.load_standard_spice_kernels()

# Set simulation start epoch.
simulation_start_epoch = 0.0

# Set numerical integration fixed step size.
fixed_step_size = 1.0

# Set simulation end epoch.
simulation_end_epoch = constants.JULIAN_DAY

17.3 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%%timeit

###########################################################################
# CREATE ENVIRONMENT ######################################################
###########################################################################

# Create body objects.
bodies_to_create = ["Earth"]

body_settings = simulation_setup.get_default_body_settings(bodies_to_create)

body_settings["Earth"].ephemeris_settings = simulation_setup.ConstantEphemerisSettings(
    np.zeros(6))

body_settings["Earth"].rotation_model_settings.reset_original_frame("ECLIPJ2000")

# Create Earth Object.
bodies = simulation_setup.create_bodies(body_settings)

7.15 ms ± 18.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit

###########################################################################
# CREATE VEHICLE ##########################################################
###########################################################################

# Create vehicle objects.
bodies["Delfi-C3"] = simulation_setup.Body()

881 ns ± 24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%%timeit

###########################################################################
# FINALIZE BODIES #########################################################
###########################################################################

simulation_setup.set_global_frame_body_ephemerides(bodies, "SSB",
                                                   "ECLIPJ2000")

634 ns ± 2.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [12]:
%%timeit

###########################################################################
# CREATE ACCELERATIONS ####################################################
###########################################################################

# Define bodies that are propagated.
bodies_to_propagate = ["Delfi-C3"]

# Define central bodies.
central_bodies = ["Earth"]

# Define accelerations acting on Delfi-C3.
accelerations_of_delfi_c3 = dict(
    Earth=[simulation_setup.Acceleration.point_mass_gravity()]
)

# Create global accelerations dictionary.
accelerations = {"Delfi-C3": accelerations_of_delfi_c3}

# Create acceleration models.
acceleration_models = simulation_setup.create_acceleration_models_dict(
    bodies, accelerations, bodies_to_propagate, central_bodies)

4.04 µs ± 27.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%%timeit

###########################################################################
# CREATE PROPAGATION SETTINGS #############################################
###########################################################################

# Set initial conditions for the Asterix satellite that will be
# propagated in this simulation. The initial conditions are given in
# Keplerian elements and later on converted to Cartesian elements.

# Set Keplerian elements for Delfi-C3
earth_gravitational_parameter = bodies[
    "Earth"].gravity_field_model.get_gravitational_parameter()

# REVISED CONTEMPORARY DESIGN.
system_initial_state = elements.keplerian2cartesian(
    mu=earth_gravitational_parameter,
    a=7500.0E3,
    ecc=0.1,
    inc=np.deg2rad(85.3),
    raan=np.deg2rad(23.4),
    argp=np.deg2rad(235.7),
    nu=np.deg2rad(139.87))

# Create propagation settings.
propagator_settings = propagators.TranslationalStatePropagatorSettings(
    central_bodies,
    acceleration_models,
    bodies_to_propagate,
    system_initial_state,
    simulation_end_epoch
)
# Create numerical integrator settings.
integrator_settings = numerical_integrators.IntegratorSettings(
    numerical_integrators.AvailableIntegrators.rungeKutta4,
    simulation_start_epoch,
    fixed_step_size
)

21 µs ± 541 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
%%timeit

###########################################################################
# PROPAGATE ORBIT #########################################################
###########################################################################

# Create simulation object and propagate dynamics.
dynamics_simulator = propagators.SingleArcDynamicsSimulator(
    bodies, integrator_settings, propagator_settings, True)
result = dynamics_simulator.get_equations_of_motion_numerical_solution()

252 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
###########################################################################
# PRINT INITIAL AND FINAL STATES ##########################################
###########################################################################

print(
    f"""
Single Earth-Orbiting Satellite Example.
The initial position vector of Delfi-C3 is [km]: \n{
    result[simulation_start_epoch][:3] / 1E3} 
The initial velocity vector of Delfi-C3 is [km]: \n{
    result[simulation_start_epoch][3:] / 1E3}
After {simulation_end_epoch} seconds the position vector of Delfi-C3 is [km]: \n{
    result[simulation_end_epoch][:3] / 1E3}
And the velocity vector of Delfi-C3 is [km]: \n{
    result[simulation_start_epoch][3:] / 1E3}
    """
)


Single Earth-Orbiting Satellite Example.
The initial position vector of Delfi-C3 is [km]: 
[7037.48400133 3238.05901792 2150.7241875 ] 
The initial velocity vector of Delfi-C3 is [km]: 
[-1.46565763 -0.04095839  6.62279761]
After 86400.0 seconds the position vector of Delfi-C3 is [km]: 
[-4560.45411243 -1438.31828479  5973.9910792 ]
And the velocity vector of Delfi-C3 is [km]: 
[-1.46565763 -0.04095839  6.62279761]
    
